In [2]:
pip install polars scikit-learn numpy pandas torch

  Using cached scikit_learn-1.7.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached markupsafe-3.0.3-cp312-cp312-win_amd64.whl.metadata (2.8 kB)
   ---------------------------------------- 0.0/783

In [1]:
import polars as pl
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
print("polars version:", pl.__version__)
print("has groupby_rolling:", hasattr(pl.DataFrame, "groupby_rolling"))
print("has group_by_rolling:", hasattr(pl.DataFrame, "group_by_rolling"))
print("has rolling:", hasattr(pl.DataFrame, "rolling"))
print("LazyFrame has rolling:", hasattr(pl.LazyFrame, "rolling"))

polars version: 1.35.2
has groupby_rolling: False
has group_by_rolling: False
has rolling: True
LazyFrame has rolling: True


In [3]:
#Clean up 

df = pl.read_csv("credit_card_transactions-ibm_v2.csv/credit_card_transactions-ibm_v2.csv")
df = df.sample(fraction=0.5, seed=42)
df.head(n = 10)

df = df.with_columns(
    pl.col("Merchant State").fill_null("ONLINE").alias("Merchant State")
)

df = df.with_columns([
    pl.col("Merchant State").fill_null(pl.lit("ONLINE")).alias("Merchant State"),
    pl.col("Zip").cast(pl.Utf8).alias("Zip_str"),
])

# 2) Conditional Zip_str rules (use pl.lit for string literals)
df = df.with_columns([
    pl.when(pl.col("Merchant State") == "ONLINE")
      .then(pl.lit("ONLINE"))
      .when(pl.col("Zip_str").is_null() & (pl.col("Merchant State") != "ONLINE"))
      .then(pl.lit("NON_US"))
      .otherwise(pl.col("Zip_str"))
      .alias("Zip_str")
])

dt_expr = (
    pl.col("Year").cast(pl.Utf8)
    + "-" + pl.col("Month").cast(pl.Utf8).str.zfill(2)
    + "-" + pl.col("Day").cast(pl.Utf8).str.zfill(2)
    + " " + pl.col("Time").cast(pl.Utf8)
)
df = df.with_columns([
    dt_expr.str.strptime(pl.Datetime, "%Y-%m-%d %H:%M").alias("DateTime")
])

# 5) Extract Date and Hour, build User_card, truncate Zip_str
df = df.with_columns([
    pl.col("DateTime").dt.date().alias("Date"),
    pl.col("DateTime").dt.hour().alias("Hour"),
    (pl.col("User").cast(pl.Utf8) + "_" + pl.col("Card").cast(pl.Utf8)).alias("User_card"),
    pl.col("Zip_str").str.slice(0, 3).alias("Zip_str"),
])

df = df.with_columns([
    pl.col("Is Fraud?")
      .cast(pl.Utf8)
      .replace({"Yes": 1, "No": 0}, default=None)
      .cast(pl.Int8)               # small int is fine
      .alias("Is Fraud?")
])

df = df.with_columns([
    pl.col("Amount").cast(pl.Utf8)
        .str.replace_all(r"[^0-9\.\-]", "")   
        .replace('', None)
        .cast(pl.Float64)                    
        .alias("Amount")
])


C:\Users\tsh_a\AppData\Local\Temp\ipykernel_29836\4015123339.py:47: DeprecationWarning: the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)
  .replace({"Yes": 1, "No": 0}, default=None)


In [4]:
df

User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Zip_str,DateTime,Date,Hour,User_card
i64,i64,i64,i64,i64,str,f64,str,i64,str,str,f64,i64,str,i8,str,datetime[μs],date,i8,str
1608,0,2002,12,4,"""06:21""",40.74,"""Swipe Transaction""",7140927389389817014,"""Sydney""","""Australia""",null,4121,null,0,"""NON""",2002-12-04 06:21:00,2002-12-04,6,"""1608_0"""
658,3,2016,12,27,"""20:00""",245.82,"""Chip Transaction""",-1818391577477071519,"""Mount Kisco""","""NY""",10549.0,5912,null,0,"""105""",2016-12-27 20:00:00,2016-12-27,20,"""658_3"""
1962,0,2011,3,20,"""14:35""",36.84,"""Swipe Transaction""",1913477460590765860,"""Jerome""","""ID""",83338.0,5300,null,0,"""833""",2011-03-20 14:35:00,2011-03-20,14,"""1962_0"""
1389,0,2009,1,27,"""22:36""",45.55,"""Swipe Transaction""",-4927381308738947637,"""Albuquerque""","""NM""",87120.0,5812,null,0,"""871""",2009-01-27 22:36:00,2009-01-27,22,"""1389_0"""
1571,4,2008,6,11,"""14:54""",3.9,"""Swipe Transaction""",-800837841896003671,"""Oceanside""","""CA""",92057.0,5921,null,0,"""920""",2008-06-11 14:54:00,2008-06-11,14,"""1571_4"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1562,1,2002,8,30,"""05:50""",20.0,"""Swipe Transaction""",-4282466774399734331,"""Dallas""","""TX""",75227.0,4829,null,0,"""752""",2002-08-30 05:50:00,2002-08-30,5,"""1562_1"""
1890,1,2006,7,29,"""09:54""",33.02,"""Swipe Transaction""",-3345936507911876459,"""Cedar Rapids""","""IA""",52402.0,7538,null,0,"""524""",2006-07-29 09:54:00,2006-07-29,9,"""1890_1"""
1115,1,2012,6,20,"""10:56""",6.6,"""Swipe Transaction""",-727612092139916043,"""Rochester""","""MN""",55901.0,5411,null,0,"""559""",2012-06-20 10:56:00,2012-06-20,10,"""1115_1"""


In [5]:
baseline_y = df.select([
    pl.col("Is Fraud?")
])

baseline_X = df.drop("Is Fraud?")

train_X, test_X, train_y, test_y = train_test_split(
    baseline_X, baseline_y, test_size=0.2, random_state=42
)

In [8]:
import numpy as np
import zipfile
from pathlib import Path

tmp_dir = Path("baseline_splits")
train_X.write_csv(tmp_dir / "train_X.csv")
test_X.write_csv(tmp_dir / "test_X.csv")
train_y.write_csv(tmp_dir / "train_y.csv")
test_y.write_csv(tmp_dir / "test_y.csv")


In [32]:
cards = pl.read_csv("sd254_cards.csv")
users = pl.read_csv("sd254_users.csv")

In [33]:
users = users.with_columns(
    pl.arange(0, users.height).alias("User")
)

cards = cards.with_columns([
    (pl.col("User").cast(pl.Utf8) + "_" + pl.col("CARD INDEX").cast(pl.Utf8)).alias("User_card")
    ])

users = users.select(["State", "FICO Score", "Yearly Income - Person", "Total Debt", "Num Credit Cards", "User"])
cards = cards.select(["User_card", "Card Brand", "Credit Limit", "Card on Dark Web"])

users = users.with_columns([
    pl.col("Total Debt").cast(pl.Utf8)
        .str.replace_all(r"[^0-9\.\-]", "")   
        .replace('', None)
        .cast(pl.Float64)                    
        .alias("Total Debt")
])

users = users.with_columns([
    pl.col("Yearly Income - Person").cast(pl.Utf8)
        .str.replace_all(r"[^0-9\.\-]", "")   
        .replace('', None)
        .cast(pl.Float64)                    
        .alias("Yearly Income - Person")
])

cards = cards.with_columns([
    pl.col("Credit Limit").cast(pl.Utf8)
        .str.replace_all(r"[^0-9\.\-]", "")   
        .replace('', None)
        .cast(pl.Float64)                    
        .alias("Credit Limit")
])

In [34]:
Ctrain_X = train_X.join(users, on="User", how="left").join(cards, on="User_card", how="left")
Ctest_X = test_X.join(users, on="User", how="left").join(cards, on="User_card", how="left")


In [35]:
Ctrain_X

User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Zip_str,DateTime,Date,Hour,User_card,State,FICO Score,Yearly Income - Person,Total Debt,Num Credit Cards,Card Brand,Credit Limit,Card on Dark Web
i64,i64,i64,i64,i64,str,f64,str,i64,str,str,f64,i64,str,str,datetime[μs],date,i8,str,str,i64,f64,f64,i64,str,f64,str
716,0,2011,12,4,"""13:53""",9.07,"""Swipe Transaction""",6780853441840436625,"""Fremont""","""CA""",94536.0,7210,null,"""945""",2011-12-04 13:53:00,2011-12-04,13,"""716_0""","""CA""",798,68765.0,75608.0,5,"""Mastercard""",39751.0,"""No"""
835,3,2012,8,18,"""10:41""",52.6,"""Swipe Transaction""",6042526206085641250,"""Bronx""","""NY""",10456.0,5912,null,"""104""",2012-08-18 10:41:00,2012-08-18,10,"""835_3""","""NY""",712,25434.0,45540.0,4,"""Mastercard""",10500.0,"""No"""
1761,0,2009,8,22,"""21:03""",15.85,"""Swipe Transaction""",6246550243095942766,"""Cocoa""","""FL""",32926.0,5921,null,"""329""",2009-08-22 21:03:00,2009-08-22,21,"""1761_0""","""FL""",644,12796.0,22192.0,3,"""Visa""",9661.0,"""No"""
1315,1,2017,8,6,"""08:23""",10.7,"""Chip Transaction""",4722913068560264812,"""Perrysburg""","""OH""",43551.0,5411,null,"""435""",2017-08-06 08:23:00,2017-08-06,8,"""1315_1""","""OR""",680,38194.0,1674.0,3,"""Mastercard""",9707.0,"""No"""
1809,1,2011,10,15,"""19:51""",58.14,"""Swipe Transaction""",4123806141501734555,"""Billings""","""MT""",59102.0,4900,null,"""591""",2011-10-15 19:51:00,2011-10-15,19,"""1809_1""","""MT""",695,41249.0,42268.0,2,"""Mastercard""",16187.0,"""No"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
748,1,2015,12,6,"""00:56""",-350.0,"""Chip Transaction""",1715299929786123066,"""Lemoore""","""CA""",93245.0,3722,null,"""932""",2015-12-06 00:56:00,2015-12-06,0,"""748_1""","""CA""",702,51465.0,0.0,2,"""Mastercard""",12200.0,"""No"""
1093,0,2018,9,8,"""15:38""",12.12,"""Chip Transaction""",8438446124682714015,"""Cairo""","""IL""",62914.0,5541,null,"""629""",2018-09-08 15:38:00,2018-09-08,15,"""1093_0""","""IL""",705,34311.0,61776.0,2,"""Mastercard""",72.0,"""No"""
1886,2,2014,5,28,"""11:38""",186.0,"""Swipe Transaction""",-6161792371494728879,"""Nassau""","""The Bahamas""",null,3389,null,"""NON""",2014-05-28 11:38:00,2014-05-28,11,"""1886_2""","""PA""",742,37579.0,0.0,5,"""Visa""",10700.0,"""No"""


In [36]:
import polars as pl
import numpy as np
import math
from typing import List as TList

# ---------------- helpers ----------------
def estimate_kappa_from_R_scalar(R: float, eps: float = 1e-12, max_kappa: float = 1e8) -> float:
    """
    Robust approximation for kappa from resultant length R (R in [0,1]).
    Avoids division by zero / numerical explosion when R -> 1 by clamping.
    Returns a small positive value for R ~ 0 and a large finite value for R ~ 1.
    """
    # sanitize input
    if R is None or not np.isfinite(R):
        return 1e-8
    # clamp into [0, 1)
    if R <= 0.0:
        return 1e-8
    if R >= 1.0 - eps:
        # R extremely close to 1 -> concentration effectively infinite; return a large finite kappa
        return float(max_kappa)

    # standard approximate formulas (Mardia & Jupp / Best & Fisher)
    if R < 0.53:
        k = 2 * R + R**3 + (5 * R**5) / 6.0
    elif R < 0.85:
        k = -0.4 + 1.39 * R + 0.43 / (1.0 - R)
    else:
        denom = (R**3 - 4 * R**2 + 3 * R)
        # protect denom from being tiny/zero
        if abs(denom) < 1e-12:
            return float(max_kappa)
        k = 1.0 / denom

    # final safety: ensure finite and positive
    if not np.isfinite(k) or k <= 0:
        return 1e-8
    return float(k)

def pct_last(lst):
    if not lst:
        return 0.0
    last = lst[-1]
    cnt_le = sum(1 for v in lst if v <= last)
    return float(cnt_le) / float(len(lst))

def card_stats_from_angles(angles):
    """
    angles: list or 1D-array of radian angles
    returns: (mu, R, kappa) with robust guarding
    """
    if angles is None or len(angles) == 0:
        return 0.0, 0.0, 1e-8
    ang = np.asarray(angles, dtype=float)
    # protect against all-nan or empty effective
    if ang.size == 0 or not np.isfinite(ang).any():
        return 0.0, 0.0, 1e-8

    comp = np.mean(np.exp(1j * ang))
    mu = float(np.angle(comp))
    R = float(np.abs(comp))
    # clamp R to [0, 1)
    R = max(0.0, min(R, 1.0 - 1e-12))
    kappa = estimate_kappa_from_R_scalar(R)
    return mu, R, kappa

def process_df (df):

    df = df.with_columns([
        (pl.col("Merchant State") != pl.col("State")).alias("merchant_state_diff")
    ])


    df = df.with_columns([
        pl.col("Amount").cast(pl.Float64).alias("Amount"),
        (pl.col("Amount") < 0).cast(pl.Int8).alias("amount_is_refund"),  # useful flag
        (
            pl.when(pl.col("Amount") == 0)
            .then(0.0)
            .otherwise(pl.col("Amount").abs().log1p()
                        * pl.when(pl.col("Amount") < 0).then(-1.0).otherwise(1.0))
        ).alias("amount_log")  
    ])


    # hour angle and sin/cos
    sec_of_day = (pl.col("DateTime").dt.hour() * 3600 + pl.col("DateTime").dt.minute() * 60 + pl.col("DateTime").dt.second())
    df = df.with_columns([
        sec_of_day.alias("sec_of_day"),
        (2 * math.pi * sec_of_day / 86400.0).alias("hour_angle"),
    ])
    df = df.with_columns([
        pl.col("hour_angle").map_elements(lambda x: float(math.sin(x))).alias("hour_sin"),
        pl.col("hour_angle").map_elements(lambda x: float(math.cos(x))).alias("hour_cos"),
    ])

    # von Mises: compute card-level mu,R,kappa from angle list then broadcast
    angles = df["hour_angle"].to_list()
    mu, R, kappa = card_stats_from_angles(angles)
    df = df.with_columns([
        pl.lit(mu).alias("card_mu"),
        pl.lit(R).alias("card_R"),
        pl.lit(kappa).alias("card_kappa"),
    ])

    # von-mises likelihood (unnormalized)
    df = df.with_columns([
        pl.struct(["card_kappa", "card_mu", "hour_angle"]).map_elements(
            lambda s: float(np.exp(s["card_kappa"] * math.cos(s["hour_angle"] - s["card_mu"])))
        ).alias("von_mises_likelihood_card")
    ])


    # tidy up per-card
    drop_cols = ["card_mu", "card_R", "sec_of_day"]
    for c in drop_cols:
        if c in df.columns:
            df = df.drop(c)

    return df

Ctrain_X = process_df(Ctrain_X)
Ctest_X = process_df(Ctest_X)


In [38]:
from pathlib import Path

tmp_dir = Path("challenger_splits")
Ctrain_X.write_csv(tmp_dir / "train_X.csv")
Ctest_X.write_csv(tmp_dir / "test_X.csv")
train_y.write_csv(tmp_dir / "train_y.csv")
test_y.write_csv(tmp_dir / "test_y.csv")


In [37]:
Ctrain_X.head(n=10)

User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Zip_str,DateTime,Date,Hour,User_card,State,FICO Score,Yearly Income - Person,Total Debt,Num Credit Cards,Card Brand,Credit Limit,Card on Dark Web,merchant_state_diff,amount_is_refund,amount_log,hour_angle,hour_sin,hour_cos,card_kappa,von_mises_likelihood_card
i64,i64,i64,i64,i64,str,f64,str,i64,str,str,f64,i64,str,str,datetime[μs],date,i8,str,str,i64,f64,f64,i64,str,f64,str,bool,i8,f64,f64,f64,f64,f64,f64
716,0,2011,12,4,"""13:53""",9.07,"""Swipe Transaction""",6780853441840436625,"""Fremont""","""CA""",94536.0,7210,null,"""945""",2011-12-04 13:53:00,2011-12-04,13,"""716_0""","""CA""",798,68765.0,75608.0,5,"""Mastercard""",39751.0,"""No""",false,0,2.309561,-1.354666,-0.976735,0.214451,0.437183,1.285982
835,3,2012,8,18,"""10:41""",52.6,"""Swipe Transaction""",6042526206085641250,"""Bronx""","""NY""",10456.0,5912,null,"""104""",2012-08-18 10:41:00,2012-08-18,10,"""835_3""","""NY""",712,25434.0,45540.0,4,"""Mastercard""",10500.0,"""No""",false,0,3.981549,-2.155191,-0.834046,-0.551694,0.437183,0.921731
1761,0,2009,8,22,"""21:03""",15.85,"""Swipe Transaction""",6246550243095942766,"""Cocoa""","""FL""",32926.0,5921,null,"""329""",2009-08-22 21:03:00,2009-08-22,21,"""1761_0""","""FL""",644,12796.0,22192.0,3,"""Visa""",9661.0,"""No""",false,0,2.824351,0.726348,0.664144,0.747605,0.437183,1.208288
1315,1,2017,8,6,"""08:23""",10.7,"""Chip Transaction""",4722913068560264812,"""Perrysburg""","""OH""",43551.0,5411,null,"""435""",2017-08-06 08:23:00,2017-08-06,8,"""1315_1""","""OR""",680,38194.0,1674.0,3,"""Mastercard""",9707.0,"""No""",true,0,2.459589,2.101667,0.862366,-0.506285,0.437183,0.704784
1809,1,2011,10,15,"""19:51""",58.14,"""Swipe Transaction""",4123806141501734555,"""Billings""","""MT""",59102.0,4900,null,"""591""",2011-10-15 19:51:00,2011-10-15,19,"""1809_1""","""MT""",695,41249.0,42268.0,2,"""Mastercard""",16187.0,"""No""",false,0,4.079908,0.207403,0.20592,0.978569,0.437183,1.432991
797,2,2012,11,18,"""17:07""",20.5,"""Swipe Transaction""",-5776107283644035423,"""Winder""","""GA""",30680.0,5813,null,"""306""",2012-11-18 17:07:00,2012-11-18,17,"""797_2""","""GA""",753,37056.0,75730.0,4,"""Mastercard""",21895.0,"""No""",false,0,3.068053,-0.322013,-0.316477,0.9486,0.437183,1.546445
1152,1,2011,6,17,"""08:13""",3.44,"""Swipe Transaction""",-727612092139916043,"""Indianapolis""","""IN""",46239.0,5411,null,"""462""",2011-06-17 08:13:00,2011-06-17,8,"""1152_1""","""IN""",830,49412.0,42378.0,3,"""Mastercard""",27594.0,"""No""",false,0,1.490654,2.095268,0.865589,-0.500756,0.437183,0.705972
1228,3,2013,6,20,"""22:22""",55.56,"""Swipe Transaction""",-5843779458842238113,"""Chicago""","""IL""",60636.0,5812,null,"""606""",2013-06-20 22:22:00,2013-06-20,22,"""1228_3""","""MD""",692,62791.0,193215.0,4,"""Mastercard""",39870.0,"""No""",true,0,4.035302,0.996583,0.83962,0.543174,0.437183,1.080176
980,2,2003,5,7,"""14:57""",64.19,"""Swipe Transaction""",8011310777263965322,"""Novi""","""MI""",48375.0,5912,null,"""483""",2003-05-07 14:57:00,2003-05-07,14,"""980_2""","""MI""",509,62103.0,33545.0,4,"""Visa""",16722.0,"""No""",false,0,4.177306,-1.094031,-0.888484,0.458908,0.437183,1.398186
